# Pandas Toolkit Series (Part 3) - Reshaping & Restructuring DataFrames

**INST447 Data Source and Manipulation**  
Author: **Wei Ai** (aiwei@umd.edu)  
University of Maryland

---

In Parts 1 and 2, we covered data selection, filtering, transformation, aggregation, groupby operations, and joins. Today we'll explore powerful techniques for **reshaping and restructuring** data:

- **Melt**: Converting wide data to long format (unpivoting)
- **Pivot**: Converting long data to wide format 
- **Pivot Tables**: Creating cross-tabulated summaries with aggregation
- **Explode**: Expanding list-like values into separate rows

These operations are essential for preparing data for analysis and visualization. We'll continue using flight-related datasets to explore these concepts.

In [1]:
import pandas as pd
import numpy as np

# Create a quarterly flight statistics dataset (wide format)
quarterly_stats = pd.DataFrame({
    'airline': ['United', 'Delta', 'Southwest', 'American'],
    'Q1_flights': [150, 180, 200, 165],
    'Q2_flights': [175, 190, 210, 170],
    'Q3_flights': [200, 195, 220, 180],
    'Q4_flights': [180, 185, 205, 175]
})

print("Wide format - Each quarter is a separate column:")
quarterly_stats

Wide format - Each quarter is a separate column:


,airline,Q1_flights,Q2_flights,Q3_flights,Q4_flights
0,United,150,175,200,180
1,Delta,180,190,195,185
2,Southwest,200,210,220,205
3,American,165,170,180,175


## 1. Melt - From Wide to Long Format

**What is "wide" vs "long" format?**

- **Wide format**: Multiple columns represent different measurements or time periods
- **Long format**: A single column for variable names and another for values

**Why would we want to convert wide to long?** 
- Long format is often required for certain visualizations (especially in libraries like seaborn)
- It's easier to perform groupby operations on long format data
- Many statistical models expect long format

In [ ]:
quarterly_stats.melt(id_vars=['airline']).

,airline,variable,value
0,United,Q1_flights,150
1,Delta,Q1_flights,180
2,Southwest,Q1_flights,200
3,American,Q1_flights,165
4,United,Q2_flights,175
5,Delta,Q2_flights,190
6,Southwest,Q2_flights,210
7,American,Q2_flights,170
8,United,Q3_flights,200
9,Delta,Q3_flights,195


In [2]:
# Basic melt - convert all quarter columns to long format
quarterly_long = quarterly_stats.melt(
    id_vars=['airline'],           # Column(s) to keep as identifier
    var_name='quarter',             # Name for the new "variable" column
    value_name='flight_count'       # Name for the new "value" column
)

print("Long format - All quarters in a single column:")
quarterly_long

Long format - All quarters in a single column:


,airline,quarter,flight_count
0,United,Q1_flights,150
1,Delta,Q1_flights,180
2,Southwest,Q1_flights,200
3,American,Q1_flights,165
4,United,Q2_flights,175
5,Delta,Q2_flights,190
6,Southwest,Q2_flights,210
7,American,Q2_flights,170
8,United,Q3_flights,200
9,Delta,Q3_flights,195


In [6]:
# What if we want to clean up the quarter names?
quarterly_long['quarter'] = quarterly_long['quarter'].str.replace('_flights', '')
quarterly_long

,airline,quarter,flight_count
0,United,Q1,150
1,Delta,Q1,180
2,Southwest,Q1,200
3,American,Q1,165
4,United,Q2,175
5,Delta,Q2,190
6,Southwest,Q2,210
7,American,Q2,170
8,United,Q3,200
9,Delta,Q3,195


In [7]:
# Now we can easily do operations like finding max flights per airline
quarterly_long.groupby('airline')['flight_count'].max()

airline
American     180
Delta        195
Southwest    220
United       200
Name: flight_count, dtype: int64

### Selective Melting

**What if we only want to melt specific columns?**

In [8]:
# Create a more complex dataset with multiple metrics
airline_metrics = pd.DataFrame({
    'airline': ['United', 'Delta', 'Southwest'],
    'Q1_flights': [150, 180, 200],
    'Q1_revenue': [4500, 5400, 5000],
    'Q2_flights': [175, 190, 210],
    'Q2_revenue': [5250, 5700, 5250]
})

airline_metrics

,airline,Q1_flights,Q1_revenue,Q2_flights,Q2_revenue
0,United,150,4500,175,5250
1,Delta,180,5400,190,5700
2,Southwest,200,5000,210,5250


In [10]:
# Melt only the flight columns
flights_melted = airline_metrics.melt(
    id_vars=['airline'],
    # value_vars=['Q1_flights', 'Q2_flights'],  # Specify which columns to melt
    var_name='quarter',
    value_name='flight_count'
)

flights_melted

,airline,quarter,flight_count
0,United,Q1_flights,150
1,Delta,Q1_flights,180
2,Southwest,Q1_flights,200
3,United,Q1_revenue,4500
4,Delta,Q1_revenue,5400
5,Southwest,Q1_revenue,5000
6,United,Q2_flights,175
7,Delta,Q2_flights,190
8,Southwest,Q2_flights,210
9,United,Q2_revenue,5250


## 2. Pivot - From Long to Wide Format

**Pivot is essentially the reverse of melt.** We take a long format and spread values across multiple columns.

**When would we use pivot?**
- Creating comparison tables
- Preparing data for certain types of reports
- Making data more human-readable

In [11]:
# Start with our long format data
print("Starting with long format:")
print(quarterly_long.head(8))

Starting with long format:
     airline quarter  flight_count
0     United      Q1           150
1      Delta      Q1           180
2  Southwest      Q1           200
3   American      Q1           165
4     United      Q2           175
5      Delta      Q2           190
6  Southwest      Q2           210
7   American      Q2           170


In [12]:
# Pivot back to wide format
quarterly_wide = quarterly_long.pivot(
    index='airline',        # Column to use as row index
    columns='quarter',      # Column whose values become new column names
    values='flight_count'   # Column whose values fill the cells
)

print("Back to wide format:")
quarterly_wide

Back to wide format:


quarter,Q1,Q2,Q3,Q4
airline,,,,
American,165,170,180,175
Delta,180,190,195,185
Southwest,200,210,220,205
United,150,175,200,180


In [15]:
# Reset the index to make it a regular column again
quarterly_wide_clean = quarterly_wide.reset_index()
quarterly_wide_clean

quarter,airline,Q1,Q2,Q3,Q4
0,American,165,170,180,175
1,Delta,180,190,195,185
2,Southwest,200,210,220,205
3,United,150,175,200,180


### Pivot with Multiple Values

**What if we want to track multiple metrics in a pivot table?**

In [16]:
# Create a dataset with route performance data
route_data = pd.DataFrame({
    'airline': ['United', 'United', 'Delta', 'Delta', 'Southwest', 'Southwest'],
    'route': ['BWI-ORD', 'ORD-LAX', 'BWI-ORD', 'ORD-LAX', 'BWI-ORD', 'ORD-LAX'],
    'avg_price': [289, 425, 275, 410, 199, 350],
    'avg_delay': [15, 20, 12, 18, 8, 25]
})

route_data

,airline,route,avg_price,avg_delay
0,United,BWI-ORD,289,15
1,United,ORD-LAX,425,20
2,Delta,BWI-ORD,275,12
3,Delta,ORD-LAX,410,18
4,Southwest,BWI-ORD,199,8
5,Southwest,ORD-LAX,350,25


In [27]:
# Pivot to see prices by airline and route
price_comparison = route_data.pivot(
    index='route',
    columns='airline',
    values=['avg_price', 
            'avg_delay']  # Multi-level columns
)

#price_comparison.columns = ['_'.join(col).strip() for col in price_comparison.columns.values]
# price_comparison

price_comparison.columns.values

array([('avg_price', 'Delta'), ('avg_price', 'Southwest'),
       ('avg_price', 'United'), ('avg_delay', 'Delta'),
       ('avg_delay', 'Southwest'), ('avg_delay', 'United')], dtype=object)

In [18]:
# Pivot to see delays by airline and route
delay_comparison = route_data.pivot(
    index='route',
    columns='airline',
    values='avg_delay'
)

delay_comparison

airline,Delta,Southwest,United
route,,,
BWI-ORD,12,8,15
ORD-LAX,18,25,20


## 3. Pivot Tables - Aggregation with Reshaping

**What's the difference between `.pivot()` and `.pivot_table()`?**

- `.pivot()`: Requires unique combinations of index/columns (no duplicates)
- `.pivot_table()`: Can handle duplicate combinations by aggregating them

**Pivot tables are like Excel pivot tables** - they reshape AND aggregate your data.

In [28]:
# Create a detailed flight log with multiple flights on same routes
flight_log = pd.DataFrame({
    'date': ['2024-01-15', '2024-01-22', '2024-02-08', '2024-02-10', 
             '2024-01-16', '2024-01-23', '2024-02-09', '2024-02-11'],
    'airline': ['United', 'Delta', 'Southwest', 'United',
                'United', 'Delta', 'Southwest', 'United'],
    'route': ['BWI-ORD', 'ORD-LAX', 'LAX-PHX', 'PHX-DEN',
              'BWI-ORD', 'ORD-LAX', 'LAX-PHX', 'PHX-DEN'],
    'price': [289, 425, 149, 210, 295, 420, 145, 215],
    'delay_min': [15, 20, 0, 45, 10, 18, 5, 40]
})

flight_log

,date,airline,route,price,delay_min
0,2024-01-15,United,BWI-ORD,289,15
1,2024-01-22,Delta,ORD-LAX,425,20
2,2024-02-08,Southwest,LAX-PHX,149,0
3,2024-02-10,United,PHX-DEN,210,45
4,2024-01-16,United,BWI-ORD,295,10
5,2024-01-23,Delta,ORD-LAX,420,18
6,2024-02-09,Southwest,LAX-PHX,145,5
7,2024-02-11,United,PHX-DEN,215,40


In [29]:
# If we try regular pivot with duplicate route-airline combinations, we get an error
# Uncomment to see the error:
flight_log.pivot(index='route', columns='airline', values='price')


ValueError: Index contains duplicate entries, cannot reshape

In [31]:
# Use pivot_table instead - automatically aggregates with mean
price_pivot = flight_log.pivot_table(
    index='route',
    columns='airline',
    values='price',
    aggfunc='sum'  # default is mean, but we can specify others
)

price_pivot

airline,Delta,Southwest,United
route,,,
BWI-ORD,NaN,NaN,584.0
LAX-PHX,NaN,294.0,NaN
ORD-LAX,845.0,NaN,NaN
PHX-DEN,NaN,NaN,425.0


In [32]:
# We can use different aggregation functions
delay_pivot = flight_log.pivot_table(
    index='route',
    columns='airline',
    values='delay_min',
    aggfunc='max'  # Show worst delay for each route-airline combo
)

delay_pivot

airline,Delta,Southwest,United
route,,,
BWI-ORD,NaN,NaN,15.0
LAX-PHX,NaN,5.0,NaN
ORD-LAX,20.0,NaN,NaN
PHX-DEN,NaN,NaN,45.0


In [33]:
# Multiple aggregation functions at once
comprehensive_pivot = flight_log.pivot_table(
    index='route',
    columns='airline',
    values='price',
    aggfunc=['mean', 'min', 'max', 'count']
)

comprehensive_pivot

mean                     min                     max            \
airline  Delta Southwest United  Delta Southwest United  Delta Southwest   
route                                                                      
BWI-ORD    NaN       NaN  292.0    NaN       NaN  289.0    NaN       NaN   
LAX-PHX    NaN     147.0    NaN    NaN     145.0    NaN    NaN     149.0   
ORD-LAX  422.5       NaN    NaN  420.0       NaN    NaN  425.0       NaN   
PHX-DEN    NaN       NaN  212.5    NaN       NaN  210.0    NaN       NaN   

               count                   
airline United Delta Southwest United  
route                                  
BWI-ORD  295.0   NaN       NaN    2.0  
LAX-PHX    NaN   NaN       2.0    NaN  
ORD-LAX    NaN   2.0       NaN    NaN  
PHX-DEN  215.0   NaN       NaN    2.0

In [34]:
# Add row and column totals with margins
price_with_totals = flight_log.pivot_table(
    index='route',
    columns='airline',
    values='price',
    aggfunc='mean',
    margins=True,
    margins_name='Average'  # Name for the total row/column
)

price_with_totals

airline,Delta,Southwest,United,Average
route,,,,
BWI-ORD,NaN,NaN,292.00,292.0
LAX-PHX,NaN,147.0,NaN,147.0
ORD-LAX,422.5,NaN,NaN,422.5
PHX-DEN,NaN,NaN,212.50,212.5
Average,422.5,147.0,252.25,268.5


### Multi-level Pivot Tables

**What if we want to analyze by multiple dimensions?**

In [35]:
# Add month information to our data
flight_log['month'] = pd.to_datetime(flight_log['date']).dt.month_name().str[:3]
flight_log

,date,airline,route,price,delay_min,month
0,2024-01-15,United,BWI-ORD,289,15,Jan
1,2024-01-22,Delta,ORD-LAX,425,20,Jan
2,2024-02-08,Southwest,LAX-PHX,149,0,Feb
3,2024-02-10,United,PHX-DEN,210,45,Feb
4,2024-01-16,United,BWI-ORD,295,10,Jan
5,2024-01-23,Delta,ORD-LAX,420,18,Jan
6,2024-02-09,Southwest,LAX-PHX,145,5,Feb
7,2024-02-11,United,PHX-DEN,215,40,Feb


In [36]:
# Create multi-level pivot table
multi_pivot = flight_log.pivot_table(
    index=['month', 'route'],  # Multiple row indexes
    columns='airline',
    values='price',
    aggfunc='mean'
)

multi_pivot

airline        Delta  Southwest  United
month route                            
Feb   LAX-PHX    NaN      147.0     NaN
      PHX-DEN    NaN        NaN   212.5
Jan   BWI-ORD    NaN        NaN   292.0
      ORD-LAX  422.5        NaN     NaN

## 4. Explode - Expanding List-Like Values

**What is explode used for?**

Sometimes our data contains list-like values (lists, tuples, sets) in cells. The `explode()` function creates a separate row for each element in these collections.

**When would we need this?**
- Processing data with multiple values in a single field
- Expanding tags, categories, or comma-separated values
- Analyzing nested or semi-structured data

In [37]:
# Create a dataset where flights have multiple connecting airports
flight_connections = pd.DataFrame({
    'flight_number': ['UA1247', 'DL456', 'WN2891'],
    'origin': ['BWI', 'ORD', 'LAX'],
    'destination': ['SFO', 'MIA', 'DEN'],
    'connections': [['ORD', 'DEN'], ['ATL'], ['PHX', 'LAS']],  # List of connection airports
    'total_price': [450, 380, 320]
})

print("Original data with list-like values:")
flight_connections

Original data with list-like values:


,flight_number,origin,destination,connections,total_price
0,UA1247,BWI,SFO,"[ORD, DEN]",450
1,DL456,ORD,MIA,[ATL],380
2,WN2891,LAX,DEN,"[PHX, LAS]",320


In [38]:
# Explode the connections column
exploded_connections = flight_connections.explode('connections')

print("After exploding - each connection gets its own row:")
exploded_connections

After exploding - each connection gets its own row:


,flight_number,origin,destination,connections,total_price
0,UA1247,BWI,SFO,ORD,450
0,UA1247,BWI,SFO,DEN,450
1,DL456,ORD,MIA,ATL,380
2,WN2891,LAX,DEN,PHX,320
2,WN2891,LAX,DEN,LAS,320


In [39]:
# Notice that the index is repeated - we can reset it
exploded_connections.reset_index(drop=True)

,flight_number,origin,destination,connections,total_price
0,UA1247,BWI,SFO,ORD,450
1,UA1247,BWI,SFO,DEN,450
2,DL456,ORD,MIA,ATL,380
3,WN2891,LAX,DEN,PHX,320
4,WN2891,LAX,DEN,LAS,320


### Real-world Example: Analyzing Flight Amenities

In [40]:
# Create a dataset where each flight has multiple amenities
flight_amenities = pd.DataFrame({
    'flight_number': ['UA1247', 'DL456', 'WN2891', 'AA892'],
    'airline': ['United', 'Delta', 'Southwest', 'American'],
    'amenities': [
        ['WiFi', 'Power', 'Entertainment'],
        ['WiFi', 'Power', 'Entertainment', 'Premium_Seats'],
        ['WiFi'],
        ['WiFi', 'Power']
    ],
    'price': [289, 425, 149, 198]
})

flight_amenities

,flight_number,airline,amenities,price
0,UA1247,United,"[WiFi, Power, Entertainment]",289
1,DL456,Delta,"[WiFi, Power, Entertainment, Premium_Seats]",425
2,WN2891,Southwest,[WiFi],149
3,AA892,American,"[WiFi, Power]",198


In [ ]:
# Explode to analyze amenities
exploded_amenities = flight_amenities.explode('amenities').reset_index(drop=True)
exploded_amenities

In [ ]:
# Now we can easily count which amenities are most common
exploded_amenities['amenities'].value_counts()

In [ ]:
# Or see which airlines offer which amenities most frequently
amenity_by_airline = exploded_amenities.groupby('airline')['amenities'].value_counts()
amenity_by_airline

### Exploding Multiple Columns

**What if we have multiple columns with list-like values that correspond to each other?**

In [41]:
# Create data with multiple list columns
flight_segments = pd.DataFrame({
    'booking_id': ['BK001', 'BK002', 'BK003'],
    'passenger': ['John', 'Mary', 'Steve'],
    'segments': [['BWI-ORD', 'ORD-LAX'], ['BOS-ATL'], ['LAX-PHX', 'PHX-DEN']],
    'segment_prices': [[150, 275], [267], [120, 150]]
})

flight_segments

,booking_id,passenger,segments,segment_prices
0,BK001,John,"[BWI-ORD, ORD-LAX]","[150, 275]"
1,BK002,Mary,[BOS-ATL],[267]
2,BK003,Steve,"[LAX-PHX, PHX-DEN]","[120, 150]"


In [42]:
# Explode both columns simultaneously
exploded_segments = flight_segments.explode(['segments', 'segment_prices']).reset_index(drop=True)
exploded_segments

,booking_id,passenger,segments,segment_prices
0,BK001,John,BWI-ORD,150
1,BK001,John,ORD-LAX,275
2,BK002,Mary,BOS-ATL,267
3,BK003,Steve,LAX-PHX,120
4,BK003,Steve,PHX-DEN,150


In [ ]:
# Now we can calculate total booking value
exploded_segments['segment_prices'] = exploded_segments['segment_prices'].astype(float)
booking_totals = exploded_segments.groupby('booking_id')['segment_prices'].sum()
booking_totals

## 5. Combining Reshape Operations

**Real data analysis often requires chaining multiple reshape operations together.** Let's work through a realistic example.

In [ ]:
# Create a complex dataset: monthly performance by airline
monthly_performance = pd.DataFrame({
    'airline': ['United', 'United', 'United', 'Delta', 'Delta', 'Delta'],
    'month': ['Jan', 'Feb', 'Mar', 'Jan', 'Feb', 'Mar'],
    'flights': [150, 160, 175, 180, 185, 190],
    'on_time_pct': [85, 87, 90, 88, 86, 89],
    'avg_delay': [15, 13, 10, 12, 14, 11]
})

monthly_performance

In [ ]:
# Step 1: Melt to long format for analysis
performance_long = monthly_performance.melt(
    id_vars=['airline', 'month'],
    var_name='metric',
    value_name='value'
)

performance_long

In [ ]:
# Step 2: Filter to specific metrics of interest
delay_data = performance_long[performance_long['metric'] == 'avg_delay']
delay_data

In [ ]:
# Step 3: Pivot to create comparison table
delay_comparison = delay_data.pivot(
    index='month',
    columns='airline',
    values='value'
)

delay_comparison

In [ ]:
# Step 4: Calculate which airline had better performance each month
delay_comparison['winner'] = delay_comparison.idxmin(axis=1)
delay_comparison

## Summary and Best Practices

### When to Use Each Operation:

**Melt (wide → long)**
- Preparing data for visualization (especially seaborn)
- Making data easier to group and aggregate
- Converting from "observation per column" to "observation per row"

**Pivot (long → wide)**
- Creating comparison tables
- Making data more human-readable
- Preparing data for certain types of analysis (e.g., correlation matrices)

**Pivot Table**
- When you need to reshape AND aggregate
- Creating summary reports with cross-tabulation
- Handling duplicate combinations of index/column values

**Explode**
- Expanding list-like values into separate rows
- Processing nested or semi-structured data
- Analyzing data with multiple values per field

### Key Takeaways:

1. **Melt and pivot are inverse operations** - melt goes wide→long, pivot goes long→wide
2. **Pivot table is like pivot but with aggregation** - use it when you have duplicate combinations
3. **Explode expands vertically** - one row becomes multiple rows
4. **Always check your data structure** before and after reshaping to ensure you got the expected result
5. **Chain operations together** for complex transformations

### Practice Exercise:

Try creating your own dataset with student grades across multiple subjects and semesters. Practice:
1. Converting between wide and long formats with melt/pivot
2. Creating a pivot table to see average grades by student and subject
3. Using explode if you have students taking multiple courses in a list